# Corporacion Favorita: Exploratory Data Analysis

_By [Michael Rosenberg](mailto:rosenberg.michael.m@gmail.com)._

_**Description**: Contains my exploratory work on the data provided in the [Corporacio Favorite Grocery Sales Competition](https://www.kaggle.com/c/favorita-grocery-sales-forecasting)._

_**Language Used**: [Julia](https://julialang.org)._

_Last Updated: 10/21/2017 9:30 PM EST._

In [4]:
#imports
using DataFrames

INFO: Precompiling module DataFrames.


In [6]:
testFrame = readtable("../data/preprocessed/test.csv");
trainFrame = readtable("../data/preprocessed/train.csv");

# Summary Statistics

In [12]:
print(nrow(trainFrame)," ",ncol(trainFrame))

125497040 6

We see that we have over 100M observations